In [12]:
import pandas as pd
from youtube_search import YoutubeSearch
from pytubefix import YouTube

### Load Scrapping Result

In [13]:
data = pd.read_csv('data/data.csv', sep=';')

data.head()

,nama,daerah,keyword,lirik
0,Kicir-Kicir,Jakarta,jakarta,NaN
1,Ondel Ondel,jakarta,Jakarta,NaN
2,Manuk Dadali,jawa barat,jawa barat,NaN


### Search and Download Music Function

In [18]:
def search_yt(query):
    results = YoutubeSearch(query, max_results=5).to_dict()

    for i in range(len(results)):
        results[i]['url'] = 'https://www.youtube.com' + results[i]['url_suffix']
        
    return results

def dl_video(query):
    try:
        yt = YouTube(query, 'MWEB')
        print(f'Downloading {yt.title}...')
        
        ys = yt.streams.get_audio_only()
        ys.download(mp3=True, output_path='songs', filename=yt.title)
        
        return f'/songs/{yt.title}.mp3'
    except Exception as e:
        print(e)
        return None
    

In [19]:
results = []

for index, row in data.iterrows():
    songs = search_yt(f"{row['nama']} asal {row['daerah']}")
    
    result_entry = {
        'nama': row['nama'],
        'daerah': row['daerah'],
        'keyword': row['keyword'],
        'songs': []
    }

    for song in songs:
        path = dl_video(song['url'])
        result_entry['songs'].append({
            'title': song['title'],
            'url': song['url'],
            'path': path
        })
    
    results.append(result_entry)


vtcZcesAk2I is streaming live and cannot be loaded


### Data Selection

In [21]:
df = pd.DataFrame(results)
df_songs = pd.DataFrame([song for result in results for song in result['songs']])

In [36]:
df_songs.head()

,title,url,path
0,KICIR KICIR | Lagu Daerah DKI Jakarta - Betawi...,https://www.youtube.com/watch?v=lhZf7qGZzsE&pp...,/songs/KICIR KICIR | Lagu Daerah DKI Jakarta -...
1,KARAOKE KICIR KICIR Lagu Daerah Jakarta,https://www.youtube.com/watch?v=nbO8hjzAN20&pp...,/songs/KARAOKE KICIR KICIR Lagu Daerah Jaka...
2,Kicir Kicir (Lagu Daerah Betawi),https://www.youtube.com/watch?v=CcFbCjM2jsM&pp...,/songs/Kicir Kicir (Lagu Daerah Betawi).mp3
3,senam kicir kicir lomba 2024,https://www.youtube.com/watch?v=NzWTp-eX02g&pp...,/songs/senam kicir kicir lomba 2024.mp3
4,Tari Daerah ~ Kicir-kicir DKI Jakarta |Rumah B...,https://www.youtube.com/watch?v=0EJP74A3Ws4&pp...,/songs/Tari Daerah ~ Kicir-kicir DKI Jakarta |...


In [37]:
import os

def convert_path(path):
    if path is None:
        return None
    return os.path.abspath(path)

df_songs['path'] = df_songs['path'].apply(convert_path)

In [38]:
df_songs.head()



,title,url,path
0,KICIR KICIR | Lagu Daerah DKI Jakarta - Betawi...,https://www.youtube.com/watch?v=lhZf7qGZzsE&pp...,c:\songs\KICIR KICIR | Lagu Daerah DKI Jakarta...
1,KARAOKE KICIR KICIR Lagu Daerah Jakarta,https://www.youtube.com/watch?v=nbO8hjzAN20&pp...,c:\songs\KARAOKE KICIR KICIR Lagu Daerah Ja...
2,Kicir Kicir (Lagu Daerah Betawi),https://www.youtube.com/watch?v=CcFbCjM2jsM&pp...,c:\songs\Kicir Kicir (Lagu Daerah Betawi).mp3
3,senam kicir kicir lomba 2024,https://www.youtube.com/watch?v=NzWTp-eX02g&pp...,c:\songs\senam kicir kicir lomba 2024.mp3
4,Tari Daerah ~ Kicir-kicir DKI Jakarta |Rumah B...,https://www.youtube.com/watch?v=0EJP74A3Ws4&pp...,c:\songs\Tari Daerah ~ Kicir-kicir DKI Jakarta...


#### Durasi Lagu
Masih error, path harus seperti ini:
`songs\senam kicir kicir lomba 2024.mp3`

In [40]:
from mutagen.mp3 import MP3
list_songs = []

for row in df_songs.iterrows():
    path = row[1]['path']
    if path is None:
        continue
    try:
        audio = MP3(path)
    except Exception as e:
        print(e)
        continue
    

[Errno 2] No such file or directory: 'c:\\songs\\KICIR KICIR | Lagu Daerah DKI Jakarta - Betawi | Budaya Indonesia | Dongeng Kita.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\KARAOKE KICIR KICIR    Lagu Daerah Jakarta.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\Kicir Kicir (Lagu Daerah Betawi).mp3'
[Errno 2] No such file or directory: 'c:\\songs\\senam kicir kicir lomba 2024.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\Tari Daerah ~ Kicir-kicir DKI Jakarta |Rumah Belajar Mawinsya.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\ONDEL ONDEL 💞 LAGU DAERAH DKI JAKARTA | ONDEL ONDEL BETAWI REMIX.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\ONDEL ONDEL | Lagu Daerah DKI Jakarta - Betawi | Budaya Indonesia | Dongeng Kita.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\LakonS - Ondel Ondel JKT 2015.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\Sejarah Asal Muasal Nama ONDEL - ONDEL.mp3'
[Errno 2] No such file or directory: 'c:\\songs\\Li